In [ ]:
import threading
import time

In [ ]:
def test_lock(): 
    lock = threading.Lock()
    name = 10
    def worker(): 
        lock.acquire()
        nonlocal name
        name += 1
        print(name)
        lock.release()

        with lock: 
            print (name)

    for j in range(2): 
        t = threading.Thread(target = worker)
        t.start()

    main_thread = threading.currentThread()
    for t in threading.enumerate(): 
        if t is not main_thread: 
            t.join()

In [2]:
def test_condition_variable(): 


SyntaxError: unexpected EOF while parsing (1632129759.py, line 1)

In [3]:
def test_event(): 
    '''
    event is great for getting an one-time event
    '''
    def foo(ev):
        print(f"flag: {ev.isSet()}")
        ev.wait(20) #timeout
        print(f"flag: {ev.isSet()}")

    ev = threading.Event()
    th1 = threading.Thread(name="Th1", target=foo, args=(ev,))
    th1.start()
    time.sleep(1)

In [4]:
def test_semaphore():
    """
    - Semaphore: integer shared by two processes
        - Just like a parking lot indicator with 3 available slots. The semaphore will start at 3. When a car gets in, calls acquire(), wait() sets semaphore--. When semaphore == 0, nobody can get it. 
        - Mutex is semaphore = 1 (binary semaphore)
    """
    from threading import Semaphore
    # when Semaphore is 1, it's binary
    s = Semaphore(7)
    w = 0
    def wurk():
        nonlocal s, w
        s.acquire()
        print("before incrementing: ", w)
        w += 1
        print("after incrementing: ", w)
        s.release()

    for i in range(10):
        t = threading.Thread(target=wurk, args=(), daemon=True)
        t.start()

In [5]:
def test_threadpool(): 
    def foo(i): 
        print(i)

    with concurrent.futures.ThreadPoolExecutor() as executor: 
        futures = []
        for i in range(5): 
            futures.append(executor.submit(foo, i=i))
        for future in futures: 
            future.result()

In [6]:
def test_multiple_threads_queue():
    """
    1. thread.setDaemon(True) will make a daemon thread,which automatically & immediately joins when the main thread is joined.
        - a regular non-daemon thread will have to wait until it finishes
    2. task_done() signifying one item has been processed to the queue
    3. join() waits for a thread to finish
    4. About shutdown:
        1. Common Practices
            - send a sentinel value with the message is common practice
            - Or have a special function that sets a flag to 0. 
        2. Pain: if you don't signal the thread, the thread will never know when to finish. Also, __del__ is not the way to go
            - garbage collection happens when reference count = 1. But 
                - when a daemon thread is still running, the enclosing object skips destruction, and it's garbage collected when exiting the program
                - when a non-daemon thread is still running, the main thread will hang because it will wait for the thread to finish
            - notes: 
                - x.__del__() may not be called during program exit.
                - del x doesn’t directly call x.__del__() — the former decrements the reference count for x by one, and the latter is only called when x’s reference count reaches zero.
        3. you can't forcibly kill a thread like killing a process (implemented on SIGTERM)
    """
    from queue import Queue, Empty
    from threading import Thread
    class Example:
        def __init__(self):
            self.should_run = True
            self.q = Queue()
            self.th = Thread(target = self.__work)
            self.th.start()
        def __work(self):
            while self.should_run:
                try: 
                    self.q.get(timeout=1)
                    self.q.task_done()
                except Empty:
                    print("work should_run: ", self.should_run)
                    pass
        def put(self, value):
            self.q.put(value)
        def shutdown(self):
            self.should_run = False
        def __del__(self):
            self.should_run = False
            print("del should_run: ", self.should_run)
            self.q.join()
    f = Example()
    f.put(1)
    # object() could be a sentinel value
    f.put(object())
    f.shutdown()

In [7]:
def test_daemon_thread():
    """
    1. Daemon Thread joins after the process is joined
        - shutdown: 
            1. If daemon thread has finished, join() succeeds and calls __del__ of daemon object
            2. if not, it will be garbage collected and its object will live until then, whose __del__ is not guaranteed to be called
    2. Problem: printing stuff in daemon thread is dangerous, may not be ablt to get lock for stdout at shutdown
    """
    from threading import Thread
    import queue
    class TestDaemon:
        def __init__(self):
            self.queue = queue.Queue()
            dth = Thread(target = self.daemon_func)
            dth.setDaemon(True)
            dth.start()
        def daemon_func(self):
            while True:
                time.sleep(0.01)
        def __del__(self):
            self.queue.join()

    t = TestDaemon()


In [ ]:
def test_threading_timer():
    '''
    1. Threading timer: execute a function on a different thread, after a certain timeout
        - You can cancel that as well.
    '''
    import threading 
    import time
    def print_msg(message):
        print(message)
    timer = threading.Timer(3, print_msg, args=("Heellloo", ))
    # start counting 3s
    timer.start()
    time.sleep(1)
    timer.cancel()
    print("Canceled timer") 

    # thread can only be started once.
    timer = threading.Timer(3, print_msg, args=("Heellloo", ))
    timer.start()
    print("Waiting for timer to fire") 
    time.sleep(4)
    print("Main thread will do its thing as well")